In [0]:
#https://www.tensorflow.org/tutorials/customization/custom_layers

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
import numpy as np

TensorFlow 2.x selected.


In [3]:
print(tf.test.is_gpu_available())


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


#Layers: common sets of useful operations

Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as a well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full Keras API in the `tf.keras package`, and the Keras layers are very useful when building your own models.

In [0]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))


The full list of pre-existing layers can be seen in the documentation. It includes Dense (a fully-connected layer), Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [23]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5])) #Aquí te da una shape 10,5 porque la layer que definimos anteriormente tiene una input_shape (None,5)
#Puedes cambiar el primer número pero no el segundo (porque da error)
#Cabe destacar que la shape que te da al usarse la layer es (tu_numero,10) porque en output le pusimos que sea 10.

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [29]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.5333032 ,  0.3914823 ,  0.49697274,  0.21362507, -0.46207976,
          0.616381  , -0.54130197,  0.01119667, -0.5612122 ,  0.35847408],
        [-0.01142257, -0.4288326 , -0.624809  ,  0.5817942 , -0.34349918,
         -0.6253674 , -0.34124473, -0.376337  , -0.12094665, -0.593232  ],
        [-0.31359243, -0.34983188,  0.14957154,  0.23651624, -0.36078745,
          0.21047437,  0.18214452, -0.00898325,  0.44647616, -0.61525667],
        [ 0.3921631 , -0.5455387 , -0.38252524, -0.62305444, -0.1674447 ,
          0.30603653,  0.36103022,  0.5937256 ,  0.5324742 ,  0.2729249 ],
        [-0.39583313, -0.00753295, -0.24510413,  0.14354521, -0.05374384,
          0.25097376,  0.49471754,  0.56225246, -0.33106467,  0.3598215 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [30]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.5333032 ,  0.3914823 ,  0.49697274,  0.21362507, -0.46207976,
          0.616381  , -0.54130197,  0.01119667, -0.5612122 ,  0.35847408],
        [-0.01142257, -0.4288326 , -0.624809  ,  0.5817942 , -0.34349918,
         -0.6253674 , -0.34124473, -0.376337  , -0.12094665, -0.593232  ],
        [-0.31359243, -0.34983188,  0.14957154,  0.23651624, -0.36078745,
          0.21047437,  0.18214452, -0.00898325,  0.44647616, -0.61525667],
        [ 0.3921631 , -0.5455387 , -0.38252524, -0.62305444, -0.1674447 ,
          0.30603653,  0.36103022,  0.5937256 ,  0.5324742 ,  0.2729249 ],
        [-0.39583313, -0.00753295, -0.24510413,  0.14354521, -0.05374384,
          0.25097376,  0.49471754,  0.56225246, -0.33106467,  0.3598215 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

#Implementing custom layers

The best way to implement your own layer is extending the `tf.keras.Layer` class and implementing: * __init__ , where you can do all input-independent initialization * build, where you know the shapes of the input tensors and can do the rest of the initialization * call, where you do the forward computation

Note that you don't have to wait until build is called to create your variables, you can also create them in __init__. However, the advantage of creating them in build is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in __init__ would mean that shapes required to create the variables will need to be explicitly specified.

In [0]:
#We are customizing a Dense layer
#im not 100% sure what exactly happened. it seems we just added a "kernel variable"
class MyDenseLayer(tf.keras.layers.Layer): 
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                    shape=[int(input_shape[-1]),
                                           self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)


In [38]:
layer #returns a type of <__main__.My_Function_Name

In [0]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.


In [43]:
#Note if the layer was not the custom one:
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))
print([var.name for var in layer.trainable_variables]) #no names in the layer (original)

layer = MyDenseLayer(10) #back to the custom_one.
_ = layer(tf.zeros([10, 5])) #It seems that we have to call this one too, to build the layer 
#just printing the name of the variable for each variable in the layer.
print([var.name for var in layer.trainable_variables])


[]
['my_dense_layer_4/kernel:0']


A tensorflow note: Overall code is easier to read and maintain if it uses standard layers whenever possible, as other readers will be familiar with the behavior of standard layers. If you want to use a layer which is not present in tf.keras.layers, consider filing a github issue or, even better, sending us a pull request!

#Models: Composing layers

Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut. Layers can be nested inside other layers.

Typically you inherit from `keras.Model` when you need the model methods like: `Model.fit`,`Model.evaluate`, and `Model.save` (see Custom Keras layers and models for details).

One other feature provided by `keras.Model` (instead of `keras.layers.Layer`) is that in addition to tracking variables, a `keras.Model` also tracks its internal layers, making them easier to inspect.

For example here is a ResNet block:

In [0]:
#This is some model we are customizing
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    #create a keras sequencial model? But assign your own filters and variables?
    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor) #add the input to the conv2a
    x = self.bn2a(x, training=training) #pass the result of conv2a to BatchNormalization
    x = tf.nn.relu(x) #get a relu layer

    x = self.conv2b(x) #Same 
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor #we are adding the input tensor to the value we got
    #Im not sure if this is an estandarized practice or if its just an example of customization
    return tf.nn.relu(x) #return a relu of x, The activation function value.
    #Remember (I think), relu when it does not reach a treshhold it returns 0, but after that it gets a value depending on X


block = ResnetIdentityBlock(1, [1, 2, 3]) #create a block with our function (type: <__main__.ResnetIdentityBlock)


In [46]:
block

In [49]:
_ = block(tf.zeros([1, 2, 3, 3])) 
block.layers #getting the layers of the block

In [50]:
len(block.variables)

18

In [52]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
________________________________________________

Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using `tf.keras.Sequential`:

In [56]:
#A simple sequatial model
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1), #The output_size, the strides (1,1), not sure what they are
                                                    input_shape=( #the input shape
                                                        None, None, 3)), #None, None, 3. When you create it another one is added, this is the batch size
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3])) #pass that array as input for the model



<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [57]:
my_seq.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_12 (Batc (None, None, None, 1)     4         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_13 (Batc (None, None, None, 2)     8         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_14 (Batc (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
_________________________________________________________